<a href="https://www.kaggle.com/code/bradhammond/tensorflow-hugging-face-unsupervised-baseline?scriptVersionId=115827360" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

Hello, I made this notebook as a tutorial on how to use Hugging Face models to create an unspervised baseline using Tensorflow. The accuracy is not as good as other unsupervised notebooks, but if you perfer using Tensorflow over pytorch this notebook will help  you understand how to do that. 


credit to https://www.kaggle.com/code/satyaprakashshukl/le-curriculum-recommendations for providing the PyTorch baseline as a starting point. 

In [1]:
pip install datasets transformers[sentencepiece]

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np 
import pandas as pd 
import transformers
from transformers import AutoConfig, AutoModel, AutoTokenizer
from tqdm.auto import tqdm
from datasets import Dataset
import tensorflow as tf

In [3]:
content_df = pd.read_csv("/kaggle/input/learning-equality-curriculum-recommendations/content.csv")
correlations_df = pd.read_csv("/kaggle/input/learning-equality-curriculum-recommendations/correlations.csv")
topics_df = pd.read_csv("/kaggle/input/learning-equality-curriculum-recommendations/topics.csv")
submission_df = pd.read_csv("/kaggle/input/learning-equality-curriculum-recommendations/sample_submission.csv")

In [4]:
model_path_or_name = '../input/transformers/bert-base-uncased'

# instantiate model & tokenizer
model     = AutoModel.from_pretrained(model_path_or_name)
tokenizer = AutoTokenizer.from_pretrained(model_path_or_name)

# make sure to import the hugging face models using the add data button in your notebook. If you don't and you put your
# notebook into offline mode this cell will not be able to load a model. 

Some weights of the model checkpoint at ../input/transformers/bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
tokenizer.is_fast

True

In [6]:
content_df_v2 = content_df.drop(columns = ["kind", 'id', 'language', 'copyright_holder', 'license'])

In [7]:
content_df_v2.head()

,title,description,text
0,"Sumar números de varios dígitos: 48,029+233,930","Suma 48,029+233,930 mediante el algoritmo está...",NaN
1,Trovare i fattori di un numero,Sal trova i fattori di 120.\n\n,NaN
2,Sumar curvas de demanda,Cómo añadir curvas de demanda\n\n,NaN
3,Nado de aproximação,Neste vídeo você vai aprender o nado de aproxi...,\nNado de aproximação\nSaber nadar nas ondas ...
4,geometry-m3-topic-a-overview.pdf,geometry-m3-topic-a-overview.pdf,Estándares Comunes del Estado de Nueva York\n\...


In [8]:
for _, row in tqdm(content_df_v2.iterrows(), total=len(content_df_v2)):
# goes through each row in the df and sets the varible 'title' = the value for that row. If value is missing 
# it assigns the value to be = the description or text column values.  
    title = row['title']
    if type(title) is float:
        title = row['description']
        row['title'] = title
    if type(title) is float:
        title = row['text']
        row['title'] = title

  0%|          | 0/154047 [00:00<?, ?it/s]

In [9]:
content_df_v2 = content_df_v2.drop(columns = ['description', 'text'])
content_df_v2.head()

,title
0,"Sumar números de varios dígitos: 48,029+233,930"
1,Trovare i fattori di un numero
2,Sumar curvas de demanda
3,Nado de aproximação
4,geometry-m3-topic-a-overview.pdf


In [10]:
dataset = Dataset.from_pandas(content_df_v2)
# we need to convert the df to a dataset so we can feed the data into the tokenizer

In [11]:
dataset

Dataset({
    features: ['title'],
    num_rows: 154047
})

In [12]:
def tokenize_function(examples):
    return tokenizer(examples["title"], max_length = 512,
        pad_to_max_length = True,
        truncation = True)

# Helper function to call the tokenizer on our dataset

In [13]:
tokenized_datasets = dataset.map(
    tokenize_function, batched=True, num_proc=4, remove_columns=["title"]
)

# call the tokenizer on the content data

#0:   0%|          | 0/39 [00:00<?, ?ba/s]

/opt/conda/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2307: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


#1:   0%|          | 0/39 [00:00<?, ?ba/s]

/opt/conda/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2307: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


#2:   0%|          | 0/39 [00:00<?, ?ba/s]

/opt/conda/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2307: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


#3:   0%|          | 0/39 [00:00<?, ?ba/s]

/opt/conda/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2307: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [14]:
vec = []
for k in tokenized_datasets['input_ids']:
    art = k 
    vec.append(art)
    
# stores the tokens in a list

In [15]:
vec = tf.stack(vec)

# reshape the list into a tf tensor so we can process it later. 

2023-01-08 16:17:02.430452: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [16]:
vec

<tf.Tensor: shape=(154047, 512), dtype=int32, numpy=
array([[  101,  7680,  2906, ...,     0,     0,     0],
       [  101, 19817,  7103, ...,     0,     0,     0],
       [  101,  7680,  2906, ...,     0,     0,     0],
       ...,
       [  101, 17174,  8566, ...,     0,     0,     0],
       [  101,  7842,  1997, ...,     0,     0,     0],
       [  101,  1996, 14855, ...,     0,     0,     0]], dtype=int32)>

In [17]:
sub_topic_ids = submission_df['topic_id'].tolist()
_topics_df = topics_df.query(f'id in {sub_topic_ids}')
# code that allows us to only select topics in the submissions_df, we will only be comparing the topics in the 
# submissions_df to the content_df. 

In [18]:
_topics_df

,id,title,description,channel,category,level,language,parent,has_content
0,t_00004da3a1b2,Откриването на резисторите,"Изследване на материали, които предизвикват на...",000cf7,source,4,bg,t_16e29365b50d,True
2,t_00068291e9a4,Entradas e saídas de uma função,Entenda um pouco mais sobre funções.,8e286a,source,4,pt,t_d14b6c2a2b70,True
3,t_00069b63a70a,Transcripts,NaN,6e3ba4,source,3,en,t_4054df11a74e,True
4,t_0006d41a73a8,Графики на експоненциални функции (Алгебра 2 н...,Научи повече за графиките на сложните показате...,000cf7,source,4,bg,t_e2452e21d252,True
19136,t_4054df11a74e,Flow Charts: Logical Thinking?,This lesson is focused on flow charts. It supp...,6e3ba4,source,2,en,t_acbbd893e6af,True


In [19]:
topics_df_v2 = _topics_df.drop(columns = ['id', 'channel', 'category', 'level', 'language', 'parent', 'has_content'])

In [20]:
topics_df_v2

,title,description
0,Откриването на резисторите,"Изследване на материали, които предизвикват на..."
2,Entradas e saídas de uma função,Entenda um pouco mais sobre funções.
3,Transcripts,NaN
4,Графики на експоненциални функции (Алгебра 2 н...,Научи повече за графиките на сложните показате...
19136,Flow Charts: Logical Thinking?,This lesson is focused on flow charts. It supp...


In [21]:
for _, row in tqdm(topics_df_v2.iterrows(), total=len(topics_df_v2)):
# goes through each row in the df and sets the varible 'title' = the value for that row. If value is missing 
# it assigns the value to be = the description or text values.  
    title = row['title']
    if type(title) is float:
        title = row['description']
        row['title'] = title
    if type(title) is float:
        title = 'has no title or description'
        row['title'] = title

  0%|          | 0/5 [00:00<?, ?it/s]

In [22]:
topics_df_v3 = topics_df_v2.drop(columns = ['description'])

In [23]:
topics_df_v3

,title
0,Откриването на резисторите
2,Entradas e saídas de uma função
3,Transcripts
4,Графики на експоненциални функции (Алгебра 2 н...
19136,Flow Charts: Logical Thinking?


In [24]:
dataset = Dataset.from_pandas(topics_df_v3)

In [25]:
dataset

Dataset({
    features: ['title', '__index_level_0__'],
    num_rows: 5
})

In [26]:
tokenized_topic_datasets = dataset.map(
    tokenize_function, batched=True, num_proc=4, remove_columns=["title"]
)

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

/opt/conda/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2307: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


#1:   0%|          | 0/1 [00:00<?, ?ba/s]

/opt/conda/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2307: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


#2:   0%|          | 0/1 [00:00<?, ?ba/s]

/opt/conda/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2307: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


#3:   0%|          | 0/1 [00:00<?, ?ba/s]

/opt/conda/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2307: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [27]:
tokenized_topic_datasets

Dataset({
    features: ['__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 5
})

In [28]:
vec2 = []
for k in tokenized_topic_datasets['input_ids']:
    art = k 
    vec2.append(art)

In [29]:
vec2 = tf.stack(vec2)

In [30]:
vec2

<tf.Tensor: shape=(5, 512), dtype=int32, numpy=
array([[  101,  1193, 22919, ...,     0,     0,     0],
       [  101,  4372,  6494, ...,     0,     0,     0],
       [  101, 24051,  2015, ...,     0,     0,     0],
       [  101,  1183, 16856, ...,     0,     0,     0],
       [  101,  4834,  6093, ...,     0,     0,     0]], dtype=int32)>

In [31]:
vec_float = tf.cast(vec, dtype=tf.float32)
vec2_float = tf.cast(vec2, dtype=tf.float32)

# you have to convert your tf int32 tensors to float32 because the cos_sim loss function won't accept int32.

In [32]:
predicts = []
for v2 in tqdm(vec2_float, total = len(vec2_float)):
    cos_sim = tf.keras.losses.cosine_similarity(
    vec_float,
    v2, axis = -1)
    p = " ".join([content_df.loc[s, 'id'] for s in tf.argsort(cos_sim.numpy()).numpy()[:5]])
    predicts.append(p)
    
# this loop compares the sim of each topic to each content then adds the content id of the 5 lowest values for each topic. 

  0%|          | 0/5 [00:00<?, ?it/s]

In [33]:
submission_df['content_ids'] = predicts
submission_df.head()

,topic_id,content_ids
0,t_00004da3a1b2,c_73169f863b30 c_df0c885a91a9 c_f58a3e74d892 c...
1,t_00068291e9a4,c_310140c79d3f c_e5c91d496d7a c_cb2e0f08e31f c...
2,t_00069b63a70a,c_16b3f3cf5a90 c_3e1010cf775b c_6c04a87db420 c...
3,t_0006d41a73a8,c_2bf407c7cc0d c_0981cea17a12 c_d8fd3d49b187 c...
4,t_4054df11a74e,c_d3ce47192e30 c_a2e474bc7974 c_c3e8425d2455 c...


In [34]:
submission_df.to_csv('submission.csv', index=None)